# Quark Star EoS Generator (CFL Model)

## Detailed Analysis

This script is used to generate synthetic Equation of State (EoS) curves for Quark Stars, representing the "Label 1" class in the machine learning dataset. It implements the Generalized Color-Flavor-Locked (CFL) model, which includes the effects of the superconducting gap energy and the strange quark mass.

The generation process employs a "Target-Driven Inverse Sampling" strategy. Rather than sampling microphysical parameters uniformly—which often results in low-mass stars—the script first selects a target maximum mass and then mathematically infers the necessary Bag Constant ($B$) to produce it. This ensures the dataset is efficiently populated with high-mass quark stars capable of mimicking observed neutron stars.

## Physics and Math

### Microphysics Parameters
The Generalized CFL model is characterized by three main parameters:
*   $B$: The effective Bag Constant (vacuum pressure), representing the confinement energy.
*   $\Delta$: The superconducting pairing gap energy.
*   $m_s$: The mass of the strange quark.

### Stability Window
For quark matter to be a viable candidate for compact stars (or strictly stable "Strange Matter"), the energy per baryon at zero pressure must be comparable to or lower than that of Iron/Neutrons ($939$ MeV). This imposes an upper limit on the Bag Constant, $B_{max}$, which depends on $\Delta$ and $m_s$:

$$
B_{max} \approx \frac{1}{(\hbar c)^3} \left[ \frac{3}{4\pi^2}\mu_{limit}^4 + \frac{3}{\pi^2}\left(\Delta^2 - \frac{m_s^2}{4}\right)\mu_{limit}^2 \right]
$$

where $\mu_{limit} = m_n / 3$ is the chemical potential per quark corresponding to the nucleon mass $m_n$.

### Inverse Scaling Law
The maximum mass of a quark star scales primarily with the Bag Constant. The script uses an empirical inverse-square relation to estimate the required $B$ for a target mass $M_{target}$:

$$
B_{target} \propto \frac{1}{M_{target}^2}
$$

A correction factor is applied to account for the stiffening effect of the gap parameter $\Delta$.

## Code Walkthrough

### 1. Parameter Sampling
The worker iterates to find valid EoS parameters. First, the strange quark mass ($m_s$) and gap energy ($\Delta$) are sampled uniformly from the physical ranges defined in `CONSTANTS`.

```python
# Sample from the physical prior ranges
ms_MeV = np.random.uniform(*CONSTANTS['Q_MS_RANGE'])
Delta_MeV = np.random.uniform(*CONSTANTS['Q_DELTA_RANGE'])
```

### 2. Stability Calculation
The theoretical stability limit $B_{max}$ is calculated based on the sampled $\Delta$ and $m_s$. If the calculated maximum is lower than the absolute minimum allowed $B$ (`Q_B_MIN`), the parameter set is physically impossible and is discarded.

### 3. Target-Driven Inverse Sampling
Instead of picking $B$ randomly, a target maximum mass is chosen (e.g., between 1.0 and 4.0 $M_{\odot}$). The required $B$ is inferred using an approximate scaling law derived from the Witten hypothesis.

```python
# Baseline guess assuming Delta=0
B_guess = 58.0 * (2.03 / target_m_max)**2

# Stiffness correction for Gap
delta_stiffness_factor = 1.0 + (1.5 * (Delta_MeV / 500.0))
B_guess_corrected = B_guess * delta_stiffness_factor
```
Noise is added to `B_guess_corrected` to ensure coverage of the parameter space, and the final value is clamped between `B_min` and `B_max`.

### 4. TOV Solution and Validation
The parameters are converted to geometric units, and `solve_sequence` is called with `is_quark=True`. The resulting curve is subjected to filters:
*   **Mass Filter:** The star must be stable and fall within the requested mass range.
*   **Radius Filter:** Stars with unrealistically large radii (`Q_R_MAX`) are discarded.

### 5. Feature Extraction
If the curve is valid, key features are interpolated at canonical masses (e.g., $1.4 M_{\odot}$).

```python
f_R = PchipInterpolator(c_arr[:,0], c_arr[:,1])
f_CS2 = interp1d(c_arr[:,0], c_arr[:,5], ...)

# Extract Features at 1.4 M_sun
cs2_at_14 = float(f_CS2(1.4))
r_14 = float(f_R(1.4))
```
If the star's maximum mass is below 1.4 $M_{\odot}$, these features are set to `NaN`.

### 6. Data Storage
The valid points on the stable branch are appended to the dataset with `Label 1` (Quark). The microphysics parameters ($B, \Delta, m_s$) are saved alongside the macroscopic observables for later analysis.